In [1]:
import yfinance as yf
ticker_symbol='MSFT'
stock_data=yf.download(ticker_symbol, start='2024-01-01',end='2025-10-27')
print(stock_data)

/tmp/ipython-input-1291561992.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data=yf.download(ticker_symbol, start='2024-01-01',end='2025-10-27')
[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            MSFT        MSFT        MSFT        MSFT      MSFT
Date                                                                
2024-01-02  366.105560  371.070940  362.058226  369.057139  25258600
2024-01-03  365.839020  368.464851  363.775873  364.269450  23083500
2024-01-04  363.213196  368.306911  362.453099  365.908135  20901500
2024-01-05  363.025635  367.280263  361.791693  364.229963  21004600
2024-01-08  369.876495  370.379953  364.269471  364.555724  23134000
...                ...         ...         ...         ...       ...
2025-10-20  516.789978  518.700012  513.429993  514.609985  14665600
2025-10-21  517.659973  518.690002  513.039978  517.500000  15586200
2025-10-22  520.539978  525.229980  517.710022  521.150024  18962700
2025-10-23  520.559998  523.950012  518.609985  522.460022  14023500
2025-10-24  523.609985  525.349976  520.710022  522.789978  15532400

[456 rows x 5 columns]


In [2]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Prepare the data
# Use the 'Close' price for prediction
data = stock_data['Close'].values.reshape(-1, 1)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 60 # Number of past days to consider for prediction
X, y = create_sequences(scaled_data, seq_length)

# Split data into training and testing sets (e.g., 80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape data for LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

print("Data preparation complete.")

Data preparation complete.


In [3]:
# Step 2: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=25))
model.add(Dense(units=1))

print("LSTM model built.")

LSTM model built.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# Step 3: Train the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=32)

print("Model training complete.")

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - loss: 0.0504
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0130
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.0097
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0074
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0078
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0087
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - loss: 0.0064
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 0.0067
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0055
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0051
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0052
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0057
Epoch 13/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0052
Epoch 14/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0050
Epoch 15/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0047
Epoc

In [7]:
# Step 4: Evaluate the model
from sklearn.metrics import mean_squared_error
from math import sqrt

# Get model predictions on the test set
predictions = model.predict(X_test)

# Inverse transform the predictions and actual values to original scale
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# Calculate RMSE (Root Mean Squared Error)
rmse = sqrt(mean_squared_error(y_test_actual, predictions))
print(f"Model Evaluation (RMSE): {rmse}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Model Evaluation (RMSE): 6.805470405684738


In [8]:

last_seq = scaled_data[-seq_length:]

# Reshape the data to fit the LSTM input shape
last_seq = np.reshape(last_seq, (1, last_seq.shape[0], 1))

# Predict the next day's stock price
predicted_price_scaled = model.predict(last_seq)

# Inverse transform the prediction to the original scale
predicted_price = scaler.inverse_transform(predicted_price_scaled)

print(f"Predicted next day closing price: {predicted_price[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted next day closing price: 518.0264282226562
